In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np
import pandas as pd 
import pickle

c:\Users\chile\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Leer el archivo data/df_lyrics_clean.json
df_tracks = pd.read_json('data/df_lyrics_clean.json')
# Leer el archivo data/playlist.csv
df_playlist = pd.read_csv('data/playlist.csv')

In [3]:
# Función para crear los conjuntos de entrenamiento y prueba
def create_train_test_sets(df, test_fraction=0.2):
    # Listas para almacenar los conjuntos de datos de prueba y entrenamiento
    train_list = []
    test_list = []
    
    # Iterar sobre cada pid único
    for pid, group in df.groupby('pid'):
        # Seleccionar aleatoriamente el 20% de las canciones del grupo actual
        test_indices = group.sample(frac=test_fraction).index
        test_set = group.loc[test_indices]
        train_set = group.drop(test_indices)
        
        # Agregar los resultados a las listas
        train_list.append(train_set)
        test_list.append(test_set)
    
    # Concatenar todos los grupos de entrenamiento y prueba en dos DataFrames
    train_set = pd.concat(train_list).reset_index(drop=True)
    test_set = pd.concat(test_list).reset_index(drop=True)
    
    return train_set, test_set

# Crear los conjuntos de entrenamiento y prueba
train_data, test_data = create_train_test_sets(df_playlist)


In [6]:
import torch

# Verifica si hay una GPU disponible
if torch.cuda.is_available():
    device = torch.device("cuda")  # Usa GPU
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # Usa CPU si no hay GPU disponible
    print("GPU no disponible, usando CPU.")

GPU disponible: NVIDIA GeForce RTX 2060


In [7]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Verifica si hay una GPU disponible y configúrala como dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Cargar el modelo y tokenizer de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)  # Mueve el modelo a la GPU si está disponible

# Función para obtener embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(device)  # Mueve los inputs a la GPU
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mueve los outputs de vuelta a la CPU para convertir a numpy

# Filtrar los track ids en el conjunto de entrenamiento
train_tids = train_data['tid'].unique()

# Crear un diccionario para almacenar los embeddings de entrenamiento
embeddings_dict_train = {}

# Generar embeddings solo para los tracks en el conjunto de entrenamiento
for tid in train_tids:
    track_info = df_tracks[df_tracks['tid'] == tid].iloc[0]
    content = f"{track_info['track_name']} {track_info['album_name']} {track_info['artist_name']} {track_info['lyrics']}"
    embeddings_dict_train[tid] = get_embeddings(content)

# Ahora los embeddings están almacenados en embeddings_dict_train y se han generado usando la GPU para acelerar el proceso

Using device: cuda


In [ ]:
# Guardar los embeddings junto con su track id
with open('data/embeddings_train.pkl', 'wb') as f:
    pickle.dump(embeddings_dict_train, f)


In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np
import pandas as pd 
import pickle

c:\Users\chile\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# DE ACA CARGA LOS EMBEDDINGS QUE SUBI AL ONEDRIVE

In [2]:
# Leer el pickle embeddings_train.pkl
with open('data/embeddings_train.pkl', 'rb') as f:
    embeddings_dict_train = pickle.load(f)


In [3]:
# Convertir el diccionario de embeddings a una matriz
track_ids = list(embeddings_dict_train.keys())
embeddings_matrix = np.array([embeddings_dict_train[tid] for tid in track_ids])

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
embeddings_matrix_reduced = pca.fit_transform(embeddings_matrix)

In [5]:
import os
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix, save_npz, load_npz
from joblib import Parallel, delayed
import numpy as np

# Definir la ubicación en el disco D:\ para los archivos temporales
temp_dir_d_drive = 'D:\\temp_similarity'

def compute_similarity_batch(embeddings_matrix, batch_i, batch_j, file_path):
    print(f"Calculando similitud para lotes {batch_i} y {batch_j}")  # Mensaje de depuración
    similarities = cosine_similarity(embeddings_matrix[batch_i[0]:batch_i[1]],
                                     embeddings_matrix[batch_j[0]:batch_j[1]])
    save_npz(file_path, csr_matrix(similarities))
    print(f"Similitud guardada en {file_path}")  # Mensaje de depuración

def compute_similarity_parallel_joblib(embeddings_matrix, batch_size=1000):
    n_tracks = embeddings_matrix.shape[0]

    # Crear el directorio temporal en el disco D:\
    os.makedirs(temp_dir_d_drive, exist_ok=True)

    batches = []

    for i in range(0, n_tracks, batch_size):
        end_i = min(i + batch_size, n_tracks)

        for j in range(i, n_tracks, batch_size):  # Solo calcular la mitad superior
            end_j = min(j + batch_size, n_tracks)

            # Definir el archivo de salida
            file_path = os.path.join(temp_dir_d_drive, f'similarity_{i}_{j}.npz')

            # Almacenar la información del lote para el proceso paralelo
            batches.append((embeddings_matrix, (i, end_i), (j, end_j), file_path))

    print("Iniciando procesos de cálculo de similitud...")  # Mensaje de depuración

    # Configurar y ejecutar los procesos en paralelo con joblib
    Parallel(n_jobs=-1)(delayed(compute_similarity_batch)(*batch) for batch in batches)

    print("Procesos de cálculo de similitud completados.")  # Mensaje de depuración

    return temp_dir_d_drive

# Calcular la matriz de similitud por lotes y guardar en el disco D:\
temp_dir = compute_similarity_parallel_joblib(embeddings_matrix_reduced)

print("Cálculo de similitud completado y guardado en disco.")  # Mensaje final de depuración

Iniciando procesos de cálculo de similitud...
Procesos de cálculo de similitud completados.
Cálculo de similitud completado y guardado en disco.


# DE ACA NO HE PODIDO LOGRAR HACER LA MATRIZ DE SIMILARIDAD, EL CODIGO ANTERIOR DESCARGAR TODAS LAS SIMILARIDADES

In [6]:
import os
from scipy.sparse import lil_matrix, csr_matrix, save_npz, load_npz
from joblib import Parallel, delayed

# Definir la ubicación en el disco D:\ para los archivos temporales
temp_dir_d_drive = 'D:\\temp_similarity'

def load_similarity_batch(file_path, i, end_i, j, end_j):
    if os.path.exists(file_path):
        print(f"Cargando similitudes desde {file_path}")
        similarities = load_npz(file_path)
        return (i, end_i, j, end_j, similarities)
    else:
        print(f"Archivo no encontrado: {file_path}")
        return None

def load_similarity_matrix_from_disk(temp_dir, n_tracks, batch_size=500, n_jobs=-1):
    similarity_matrix = lil_matrix((n_tracks, n_tracks), dtype=np.float32)
    print("Iniciando tareas paralelas para cargar y ensamblar la matriz de similitud desde el disco...")

    batches = []

    for i in range(0, n_tracks, batch_size):
        end_i = min(i + batch_size, n_tracks)
        for j in range(i, n_tracks, batch_size):
            end_j = min(j + batch_size, n_tracks)
            file_path = os.path.join(temp_dir, f'similarity_{i}_{j}.npz')
            batches.append((file_path, i, end_i, j, end_j))

    print(f"Total de lotes a procesar: {len(batches)}")  # Depuración: número total de lotes

    # Procesar lotes en paralelo
    results = Parallel(n_jobs=n_jobs, verbose=5)(delayed(load_similarity_batch)(*batch) for batch in batches)

    print("Tareas paralelas completadas. Ensamblando la matriz...")

    for result in results:
        if result is not None:
            i, end_i, j, end_j, similarities = result
            similarity_matrix[i:end_i, j:end_j] = similarities
            if i != j:
                similarity_matrix[j:end_j, i:end_i] = similarities.transpose()
            print(f"Lotes {i}-{end_i} y {j}-{end_j} ensamblados.")  # Depuración: ensamblaje de lotes

    print("Matriz de similitud cargada y ensamblada desde el disco.")
    return similarity_matrix.tocsr()

# Cargar la matriz de similitud desde el disco D:\
n_tracks = embeddings_matrix_reduced.shape[0]
similarity_matrix_sparse = load_similarity_matrix_from_disk(temp_dir_d_drive, n_tracks)

print("Matriz de similitud cargada desde el disco.")


Iniciando tareas paralelas para cargar y ensamblar la matriz de similitud desde el disco...
Total de lotes a procesar: 43365


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 204 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 576 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1680 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 3492 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-1)]: Done 4764 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 5874 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 6936 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 8034 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 9168 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 10805 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 12120 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 13283 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 15072 tasks  

KeyboardInterrupt: 